# McArthur Forest Fire Danger Index (FFDI) computation

Doug Richardson (CSIRO): doug.richardson@csiro.au

This notebook details how we compute the FFDI from model data (either the JRA reanalysis or ACCESS-D hindcasts) for our work. We use the formula provided in Dowdy (2018) https://journals.ametsoc.org/view/journals/apme/57/2/jamc-d-17-0167.1.xml, with the following differences:

- The drought factor is not calculated using the Keetch-Byram Drought Index (KBDI), as the procedure for the computing the KBDI is rather involved and empirical. We use 20-day accumulated rainfall instead.
- We do not calibrate our wind speed data in the same way as Dowdy (2018), who used 10-minute NWP forecasts to calibrate mid-afternoon wind speed. We instead use daily max wind speed from JRA, or daily mean wind speed from ACCESS-D (as there is no max wind speed in that data set).
- We can take relative humidity from the model directly, rather than having to estimate it using vapour pressure. We use mid-afternoon (0600 UTC) relative humidity from JRA, but Dougie uses daily mean relative humidity in his work as that's what's available in ACCESS-D. For the same reason, Dougie uses relative humidity at 1000 hPa rather than at the surface.

## FFDI variables:
- daily mean or mid-afternoon relative humidity at 2 m or 1000 hPa ($H$)
- daily max or mean 10 m wind speed ($W$)
- daily max 2 m temperature ($T$)
- drought factor ($D$)

## FFDI formula:
## $$\mathit{FFDI} = D^{0.987} \cdot \mbox{exp}(0.0338T - 0.0345H + 0.0234W + 0.243147)$$

# Code for calculating the FFDI using the JRA reanalysis

In [1]:
import xarray as xr
import numpy as np

## Load JRA data

In [3]:
jra_ds = xr.open_zarr('jra_surface_daily.zarr', consolidated=True)

In [18]:
jra_ds = jra_ds.rename({'initial_time0_hours': 'time'})

In [49]:
jra_ds.data_vars

Data variables:
    DEPR_GDS0_HTGL   (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    POT_GDS0_SFC     (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    PRES_GDS0_SFC    (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    PRMSL_GDS0_MSL   (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    RH_GDS0_HTGL     (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    SPFH_GDS0_HTGL   (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    SoilW_GDS0_ULN   (time, lv_ULN1, g0_lat_2, g0_lon_3) float32 dask.array<chunksize=(731, 1, 145, 288), meta=np.ndarray>
    TMAX_GDS4_HTGL   (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 145, 288), meta=np.ndarray>
    TMP_GDS0_HTGL    (time, g0_lat_1, g0_lon_2) float32 dask.array<chunksize=(731, 1

## Compute the drought factor, $D$

#### Get precipitation

In [20]:
jra_precip = jra_ds['TPRAT_GDS0_SFC']

#### Calculate 20-day accumulated precipitation

In [22]:
jra_precip_20 = jra_precip.rolling(time=20).sum()

#### Using the following formula, scale this quantity to lie between 0 and 10, with larger values indicating lower 20-day precipitation totals.

$$-10 \cdot \frac{x_i - \min(x)}{\max(x)-\min(x)} + 10.$$

In [23]:
df = -10 * ((jra_precip_20 - jra_precip_20.min(dim='time')) / (jra_precip_20.max(dim='time') - jra_precip_20.min(dim='time'))) + 10

## Get relative humidity, max temperature and max wind

In [ ]:
rh = jra_ds['RH_GDS0_HTGL']
tmax = jra_ds['TMAX_GDS4_HTGL']
wmax = jra_ds['WSMX_GDS4_HTGL']

#### Convert temperature from K to C

In [33]:
tmax = tmax - 273.15

#### Convert wind speed from m/s to km/h

In [34]:
wmax = wmax * 3.6

## Calculate FFDI

In [36]:
ffdi = df**0.987 * np.exp(0.0338*tmax - 0.0345*rh + 0.0234*wmax + 0.243147)